In [1]:
import pandas as pd

df = pd.read_csv('/Users/marcelosilva/Desktop/projectOne/3/C-Variable Analysis/complete_cases_dataset.csv')

In [2]:
# Show shape of the dataframe
print("Shape of the dataset:", df.shape)

# Check for NaN values in each column
print("\nNumber of NaN values in each column:")
print(df.isna().sum())

Shape of the dataset: (4339, 24)

Number of NaN values in each column:
id_anon                   0
b02_sexo                  0
b04_idade                 0
bb04_idade_da_mae         0
d01_cor                   0
h01_semanas_gravidez      0
h02_peso                  0
h03_altura                0
h04_parto                 0
j03_cor                   0
k04_prenatal_semanas      0
k05_prenatal_consultas    0
k06_peso_engravidar       0
k07_peso_final            0
k08_quilos                0
k12_tempo                 0
k13_tempo_medida          0
k15_recebeu               0
k16_liquido               0
k18_somente               0
k19_somente_medida        0
t05_altura_medida1        0
t06_altura_medida2        0
vd_zimc                   0
dtype: int64



# Data Cleaning: Handling Missing Responses in Categorical Variables

In this step, we will remove the "Don't know/No response" categories from the categorical variables in our dataset. This cleaning process is essential to:

- Improve data quality
- Focus on valid responses
- Reduce noise in our analysis
- Prepare the data for meaningful statistical analysis

The categorical variables that will be cleaned are:
- b02_sexo (Sex)
- d01_cor (Mother's skin color)
- h04_parto (Type of delivery)
- j03_cor (Child's skin color)
- k13_tempo_medida (Time measure unit)
- k15_recebeu (Received)
- k16_liquido (Liquid)
- k19_somente_medida (Only measure)
```

In [3]:
# Get list of categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns

# For each categorical column, show value counts and percentage
for column in categorical_columns:
    print(f"\nDistribution for {column}:")
    value_counts = df[column].value_counts()
    value_percentages = df[column].value_counts(normalize=True) * 100
    
    # Combine counts and percentages
    summary = pd.DataFrame({
        'Count': value_counts,
        'Percentage': value_percentages
    })
    print(summary)


Distribution for b02_sexo:
           Count  Percentage
b02_sexo                    
Masculino   2207   50.864254
Feminino    2132   49.135746

Distribution for d01_cor:
                                                    Count  Percentage
d01_cor                                                              
Parda (mulata, cabocla, cafuza, mameluca ou mes...   2267   52.247062
Branca                                               1753   40.401014
Preta                                                 292    6.729661
Amarela (origem japonesa, chinesa, coreana etc.)       19    0.437889
Indígena                                                8    0.184374

Distribution for h04_parto:
                                      Count  Percentage
h04_parto                                              
Normal                                 2179   50.218944
Cesariana de urgência (Não agendada)   1142   26.319428
Cesariana agendada (eletiva)           1018   23.461627

Distribution for j03_cor:
   

In [4]:

import numpy as np
import os

def create_clean_dataset(df, output_path):
    """
    Cria CleanDataset.csv removendo todos os casos com 'Não sabe/Não quis responder'
    """
    
    # Verificar se o diretório existe, se não, criar
    os.makedirs(output_path, exist_ok=True)
    
    print("CRIANDO CLEANDATASET.CSV")
    print("=" * 40)
    
    # Informações iniciais
    initial_count = len(df)
    print(f"📊 Dataset inicial: {initial_count:,} pacientes")
    
    # Fazer cópia do dataset
    df_clean = df.copy()
    
    # Identificar e contar casos com "Não sabe/Não quis responder"
    patterns_to_remove = [
        'Não sabe/Não quis responder',
        'Não sabe',
        'Não quis responder',
        'NS/NR'
    ]
    
    print(f"\n🔍 Procurando padrões de missing categórico:")
    
    total_cases_to_remove = set()  # Usar set para evitar duplicatas
    removal_details = {}
    
    # Verificar cada coluna categórica
    categorical_cols = df_clean.select_dtypes(include=['object']).columns
    
    for col in categorical_cols:
        col_removals = 0
        found_patterns = []
        
        for pattern in patterns_to_remove:
            # Buscar padrão (case insensitive)
            mask = df_clean[col].str.contains(pattern, case=False, na=False)
            matching_cases = df_clean[mask].index.tolist()
            
            if len(matching_cases) > 0:
                # Adicionar índices ao set de casos para remover
                total_cases_to_remove.update(matching_cases)
                col_removals += len(matching_cases)
                found_patterns.append(f"'{pattern}': {len(matching_cases)}")
        
        if found_patterns:
            removal_details[col] = {
                'patterns': found_patterns,
                'total_in_column': col_removals
            }
            print(f"   {col}: {', '.join(found_patterns)}")
    
    # Mostrar resumo dos casos a serem removidos
    total_unique_removals = len(total_cases_to_remove)
    
    if total_unique_removals == 0:
        print("✅ Nenhum caso com missing categórico encontrado!")
        df_final = df_clean
    else:
        print(f"\n✂️ Removendo casos:")
        print(f"   Total de casos únicos a remover: {total_unique_removals}")
        
        # Remover casos identificados
        df_final = df_clean.drop(index=list(total_cases_to_remove))
        
        print(f"   Casos removidos: {initial_count - len(df_final)}")
        print(f"   Casos restantes: {len(df_final):,}")
    
    # Verificação final - confirmar que não há mais missing categóricos
    print(f"\n🔬 Verificação final:")
    remaining_missing = 0
    
    for col in df_final.select_dtypes(include=['object']).columns:
        for pattern in patterns_to_remove:
            count = df_final[col].str.contains(pattern, case=False, na=False).sum()
            remaining_missing += count
    
    if remaining_missing == 0:
        print("✅ Confirmado: Nenhum missing categórico restante!")
    else:
        print(f"⚠️ Ainda existem {remaining_missing} missing categóricos!")
    
    # Definir caminho do arquivo
    filename = "CleanDataset.csv"
    full_path = os.path.join(output_path, filename)
    
    # Salvar o dataset
    print(f"\n💾 Salvando CleanDataset.csv:")
    print(f"📁 {full_path}")
    
    try:
        df_final.to_csv(full_path, index=False)
        print("✅ CleanDataset.csv salvo com sucesso!")
        
        # Informações do arquivo
        file_size = os.path.getsize(full_path) / (1024 * 1024)  # MB
        
        print(f"\n📊 CLEANDATASET.CSV - INFORMAÇÕES:")
        print("=" * 40)
        print(f"📈 Linhas: {len(df_final):,}")
        print(f"📈 Colunas: {len(df_final.columns)}")
        print(f"📈 Tamanho do arquivo: {file_size:.2f} MB")
        print(f"📈 Missing values numéricos: {df_final.isnull().sum().sum()}")
        print(f"📈 Missing values categóricos: {remaining_missing}")
        
        # Taxa de retenção
        retention_rate = (len(df_final) / initial_count) * 100
        print(f"📈 Taxa de retenção: {retention_rate:.2f}%")
        
        # Resumo da qualidade dos dados
        print(f"\n🏆 QUALIDADE DO DATASET:")
        if df_final.isnull().sum().sum() == 0 and remaining_missing == 0:
            print("✅ DATASET COMPLETAMENTE LIMPO!")
            print("   - 0 missing values numéricos")
            print("   - 0 missing values categóricos")
            print("   - 100% dados genuínos")
        else:
            print("⚠️ Dataset ainda tem alguns problemas")
        
        # Mostrar distribuição final das variáveis que tinham problemas
        problem_vars = ['k15_recebeu', 'k16_liquido']
        
        print(f"\n📋 Distribuição final das variáveis corrigidas:")
        for var in problem_vars:
            if var in df_final.columns:
                print(f"\n{var}:")
                value_counts = df_final[var].value_counts()
                for value, count in value_counts.items():
                    pct = (count / len(df_final)) * 100
                    print(f"   {value}: {count:,} ({pct:.1f}%)")
        
    except Exception as e:
        print(f"❌ Erro ao salvar CleanDataset.csv: {e}")
        return None
    
    return df_final

def validate_clean_dataset(df_clean):
    """
    Validação completa do CleanDataset
    """
    print(f"\n🔬 VALIDAÇÃO COMPLETA DO CLEANDATASET")
    print("=" * 50)
    
    # 1. Verificar shape
    print(f"📊 Shape: {df_clean.shape}")
    
    # 2. Verificar tipos de dados
    print(f"📊 Tipos de variáveis:")
    print(f"   Numéricas: {len(df_clean.select_dtypes(include=[np.number]).columns)}")
    print(f"   Categóricas: {len(df_clean.select_dtypes(include=['object']).columns)}")
    
    # 3. Verificar missing values
    total_missing = df_clean.isnull().sum().sum()
    print(f"📊 Missing values: {total_missing}")
    
    # 4. Verificar valores únicos das variáveis categóricas
    print(f"\n📋 Resumo das variáveis categóricas:")
    categorical_cols = df_clean.select_dtypes(include=['object']).columns
    
    for col in categorical_cols:
        unique_count = df_clean[col].nunique()
        most_frequent = df_clean[col].mode().iloc[0] if len(df_clean[col].mode()) > 0 else "N/A"
        print(f"   {col}: {unique_count} categorias únicas, mais frequente: '{most_frequent}'")
    
    # 5. Verificar se há padrões suspeitos restantes
    suspicious_patterns = ['999', '99', 'Não sabe', 'NS', 'NR', 'Missing']
    
    found_suspicious = False
    for col in categorical_cols:
        for pattern in suspicious_patterns:
            if df_clean[col].astype(str).str.contains(pattern, case=False, na=False).any():
                print(f"⚠️ Padrão suspeito '{pattern}' encontrado em {col}")
                found_suspicious = True
    
    if not found_suspicious:
        print(f"✅ Nenhum padrão suspeito encontrado!")
    
    # 6. Estatísticas básicas
    print(f"\n📊 Estatísticas básicas das variáveis numéricas:")
    numeric_summary = df_clean.describe()
    print(f"   Variáveis numéricas: {len(numeric_summary.columns)}")
    print(f"   Observações por variável: {numeric_summary.loc['count'].min():.0f} - {numeric_summary.loc['count'].max():.0f}")
    
    return total_missing == 0 and not found_suspicious

# EXECUTAR CRIAÇÃO DO CLEANDATASET
# =================================

output_directory = "/Users/marcelosilva/Desktop/projectOne/3/D-Variable Analysis"

print("PROCESSO DE CRIAÇÃO DO CLEANDATASET.CSV")
print("=" * 60)

# Criar CleanDataset.csv
clean_dataset = create_clean_dataset(df, output_directory)

# Validar dataset criado
if clean_dataset is not None:
    is_valid = validate_clean_dataset(clean_dataset)
    
    if is_valid:
        print(f"\n🎉 CLEANDATASET.CSV CRIADO COM SUCESSO!")
        print(f"📂 Arquivo: CleanDataset.csv")
        print(f"📍 Localização: {output_directory}")
        print(f"📊 Shape final: {clean_dataset.shape}")
        print(f"🏆 Qualidade: 100% dados limpos e completos!")
    else:
        print(f"\n⚠️ CleanDataset criado mas precisa de revisão!")
    
    # Mostrar amostra do dataset final
    print(f"\n📋 AMOSTRA DO CLEANDATASET (5 primeiras linhas):")
    print("=" * 60)
    
    # Mostrar algumas colunas representativas
    sample_cols = ['id_anon', 'b02_sexo', 'd01_cor', 'h04_parto', 'k15_recebeu', 'k16_liquido']
    available_cols = [col for col in sample_cols if col in clean_dataset.columns]
    
    if available_cols:
        print(clean_dataset[available_cols].head())
else:
    print("❌ Falha na criação do CleanDataset.csv")

PROCESSO DE CRIAÇÃO DO CLEANDATASET.CSV
CRIANDO CLEANDATASET.CSV
📊 Dataset inicial: 4,339 pacientes

🔍 Procurando padrões de missing categórico:
   k15_recebeu: 'Não sabe/Não quis responder': 35, 'Não sabe': 35, 'Não quis responder': 35
   k16_liquido: 'Não sabe/Não quis responder': 31, 'Não sabe': 31, 'Não quis responder': 31

✂️ Removendo casos:
   Total de casos únicos a remover: 52
   Casos removidos: 52
   Casos restantes: 4,287

🔬 Verificação final:
✅ Confirmado: Nenhum missing categórico restante!

💾 Salvando CleanDataset.csv:
📁 /Users/marcelosilva/Desktop/projectOne/3/D-Variable Analysis/CleanDataset.csv
✅ CleanDataset.csv salvo com sucesso!

📊 CLEANDATASET.CSV - INFORMAÇÕES:
📈 Linhas: 4,287
📈 Colunas: 24
📈 Tamanho do arquivo: 0.81 MB
📈 Missing values numéricos: 0
📈 Missing values categóricos: 0
📈 Taxa de retenção: 98.80%

🏆 QUALIDADE DO DATASET:
✅ DATASET COMPLETAMENTE LIMPO!
   - 0 missing values numéricos
   - 0 missing values categóricos
   - 100% dados genuínos

📋 Distribu